In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import * 
pd.options.display.max_columns = 1000
from time import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import * 

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/Users/abasar/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df_train = pd.read_csv("/Users/abasar/data/MNIST/mnist_train.csv", header=None)
df_test = pd.read_csv("/Users/abasar/data/MNIST/mnist_test.csv", header=None)

X_train = (df_train.iloc[:, 1:].values/255).reshape(-1, 28, 28, 1) # width, height, channel
X_test = (df_test.iloc[:, 1:].values/255).reshape(-1, 28, 28, 1)
y_train = df_train.iloc[:, 0].values
y_test = df_test.iloc[:, 0].values

In [3]:
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [4]:
Y_train = keras.utils.to_categorical(y_train)
Y_test = keras.utils.to_categorical(y_test)

In [5]:
tf.random.set_seed(1)

def build_model():
    model = keras.Sequential()
    model.add(InputLayer(input_shape=(28,28,1)))
    model.add(Conv2D(filters=32, activation="relu", kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, activation="relu", kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(rate=0.4))
    model.add(Dense(activation="relu", units=100))
    model.add(Dense(activation="softmax", units=10))
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.1)
    
    model.compile(loss=keras.losses.categorical_crossentropy
                  , optimizer=opt
                  , metrics = ["accuracy"])
    return model

model = build_model()
model.summary()

tensorboard = keras.callbacks.TensorBoard(log_dir=f"/tmp/tf/" + str(int(time())), 
                                          histogram_freq=1,
                                          write_images=True)

model.fit(x = X_train
          , y = Y_train
          , validation_data = (X_test, Y_test)
          , verbose = 1
          , epochs = 1
          , batch_size= 64
          , callbacks = [tensorboard]
         )
model.evaluate(X_test, Y_test)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               1

[2.31538462638855, 0.11349999904632568]

## Model with learning rate decay

In [8]:
tf.random.set_seed(1)

def build_model():
    model = keras.Sequential()
    model.add(Conv2D(filters=32, activation="relu", kernel_size=(5, 5), input_shape = (28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, activation="relu", kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(rate=0.8))
    model.add(Dense(activation="relu", units=100))
    model.add(Dense(activation="softmax", units=n_y))
    
    
    # initial_learning_rate * decay_rate ^ (step / decay_steps)
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
                                                initial_learning_rate=1e-2,
                                                decay_steps=10000,
                                                decay_rate=0.9)
    
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model.compile(loss=keras.losses.categorical_crossentropy
                  , optimizer=opt
                  , metrics = ["accuracy"])
    return model

model = build_model()
model.summary()

tensorboard = keras.callbacks.TensorBoard(log_dir=f"/tmp/tf/" + str(int(time())), 
                                          histogram_freq=1,
                                          write_images=True)

model.fit(x = X_train
          , y = Y_train
          , validation_data = (X_test, Y_test)
          , verbose = 1
          , epochs = 1
          , batch_size= 64
          , callbacks = [tensorboard]
         )
model.evaluate(X_test, Y_test)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)              

[0.08927067369222641, 0.9732000231742859]

## Model using data generator

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 64
drop_out = 0.4

tf.random.set_seed(1)

def build_model():
    model = keras.Sequential()
    model.add(Conv2D(filters=32, activation="relu", kernel_size=(5, 5), input_shape = (28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, activation="relu", kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(rate=0.4))
    model.add(Dense(activation="relu", units=100))
    model.add(Dense(activation="softmax", units=n_y))
    
    # initial_learning_rate * decay_rate ^ (step / decay_steps)
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
                                                initial_learning_rate=1e-2,
                                                decay_steps=10000,
                                                decay_rate=0.9)
    
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model.compile(loss=keras.losses.categorical_crossentropy
                  , optimizer=opt
                  , metrics = ["accuracy"])
    return model

model = build_model()
model.summary()

tensorboard = keras.callbacks.TensorBoard(log_dir=f"/tmp/tf/" + str(int(time())), 
                                          histogram_freq=1,
                                          write_images=True)


datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )

model.fit(datagen.flow(X_train, Y_train, batch_size=batch_size)
                    , steps_per_epoch=X_train.shape[0] // batch_size
                    , epochs=1
                    , verbose=1
                    , validation_data=(X_test, Y_test)
                    , callbacks=[tensorboard])

model.evaluate(X_test, Y_test)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)              

[0.08180491626262665, 0.9746000170707703]